In [1]:
import pandas as pd
import errno    
import os
import numpy as np
import math

'''
Author: Enrico Ceccolini
    TODO
'''

datadir = "/datasets/eurora_data/db1/"

infile_jobs_to_nodes = datadir + "job_nodes.csv"

suffix = "_5sec_"
### select an interval from
## 1 settings wholeData
#interval_comment_whole = "WholeData"

### select an interval from
## 2 settings Andrea
interval_comment = "Andrea"
start_time = pd.to_datetime('2014-03-31')
end_time = pd.to_datetime('2014-05-01')
#infile_jobs = datadir + "april_long_jobs.csv"

## 3 settings Alina
#interval_comment = "Alina"
#start_time = pd.to_datetime('2014-06-30')
#end_time = pd.to_datetime('2014-11-01')

infile_long_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_long_jobs_real_pow"
infile_short_jobs = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_short_jobs_real_pow"

nodes=['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64']

In [2]:
long_jobs = pd.read_csv(infile_long_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(long_jobs.shape[0], interval_comment))

There are 35284 long jobs in the Andrea's interval


/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Calculate the mean core consumption

In [8]:
# clean user names
long_jobs['user_short'] = long_jobs.apply(lambda row: row['user'].split('@')[0], axis=1)

In [9]:
grouped_user_data = long_jobs.groupby(['user_short', 'job_name'])['job_id_string'].nunique().reset_index(name='counts')
print("{} distinct user-jobs".format(grouped_user_data.shape[0]))

grouped_user_data['mean_overAll'] = 0.0

2871 distinct user-jobs


In [10]:
grouped_user_data

,user_short,job_name,counts,mean_overAll
0,a07smr01,A2REPO,51,0.0
1,a07smr01,ARKI,57,0.0
2,a07smr01,INIT,59,0.0
3,a07smr01,LM28_OPE,66,0.0
4,a07smr01,LM7_OPE,56,0.0
5,a07smr01,LM7_TST,48,0.0
6,a07smr01,LM_PRE,88,0.0
7,a07smr01,POSTPROCESS,53,0.0
8,a07smr01,SWAN_CAM,46,0.0
9,a07smr01,SWAN_EMR,46,0.0


In [12]:
#debug
#user_jobs = long_jobs[long_jobs['user_short'] == 'a07smr01']
#user_current_job = user_jobs[user_jobs['job_name'] == 'A2REPO']
#user_current_job



In [13]:
#debug
#user_current_job['real_pow'].sum() / user_current_job['cpu_req'].sum()

In [36]:
i = 0
for job_index, job_row in grouped_user_data.iterrows():
    user_jobs = long_jobs[long_jobs['user_short'] == job_row['user_short']]
    user_current_job = user_jobs[user_jobs['job_name'] == job_row['job_name']]
    #print(user_current_job.shape[0])
    
    if(user_current_job['cpu_req'].sum() != 0):
        mean_core_overAll = user_current_job['real_pow'].sum() / user_current_job['cpu_req'].sum()
        grouped_user_data.at[job_index, 'mean_overAll'] = mean_core_overAll

    #print("{}, {}, {}, {}".format(user_row['user'], mean_core_2_1, mean_core_2_2, mean_core_3_1))
    
    if(i % 100 == 0):
        print(i)
    i += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800


In [37]:
grouped_user_data
# Problemi: utenti che non hanno eseguito job su singoli nodi

,user_short,job_name,counts,mean_overAll
0,a07smr01,A2REPO,51,4.385339
1,a07smr01,ARKI,57,4.301715
2,a07smr01,INIT,59,4.430796
3,a07smr01,LM28_OPE,66,15.905794
4,a07smr01,LM7_OPE,56,9.239231
5,a07smr01,LM7_TST,48,18.621090
6,a07smr01,LM_PRE,88,11.946165
7,a07smr01,POSTPROCESS,53,5.077936
8,a07smr01,SWAN_CAM,46,9.859832
9,a07smr01,SWAN_EMR,46,9.930792


In [38]:
outfile_grouped_user_data =  datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_consumption_userJobs"
grouped_user_data.to_csv(outfile_grouped_user_data + ".csv")

## Predict

In [5]:


infile_mean_core_consumptions = datadir + "CPUs/" + interval_comment + "/" + interval_comment + "_mean_core_consumption_userJobs"
mean_core_consumptions = pd.read_csv(infile_mean_core_consumptions + ".csv", index_col=0)

long_jobs = pd.read_csv(infile_long_jobs + ".csv", index_col=0)
print("There are {} long jobs in the {}'s interval".format(long_jobs.shape[0], interval_comment))
# clean user names
long_jobs['user_short'] = long_jobs.apply(lambda row: row['user'].split('@')[0], axis=1)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


There are 35284 long jobs in the Andrea's interval


In [15]:
mean_core_consumptions.sort_values('counts', ascending=False)

,user_short,job_name,counts,mean_overAll
2832,szia0000,run_SMD,2882,13.961007
1387,fpetracc,cosmomc_nMin20_,1590,17.237141
2781,szia0000,Run-4e,632,9.901939
1597,jfranz00,g.job,513,24.334461
2834,tbusatta,scriptcineca_10,472,37.847148
2780,szia0000,Run-4a,319,12.867697
2838,tbusatta,scriptcineca_50,313,36.889236
2837,tbusatta,scriptcineca_40,309,41.271373
2757,spezzott,DL_POLY_multi,274,28.030871
1641,kkwapien,job,271,20.335084


In [16]:
long_jobs['pred_userJob'] = 0.0

In [17]:
# debug
#df = mean_core_consumptions[mean_core_consumptions['user_short'] == 'jfranz00']
#df = df[df['job_name'] == 'g.job']
#df['mean_overAll']

In [18]:
i = 0
for job_index, job_row in long_jobs.iterrows():
    job_user = job_row['user_short']
    job_name = job_row['job_name']
    job_n_cores_2_1 = job_row['n_cores_2_1']
    job_n_cores_2_2 = job_row['n_cores_2_2']
    job_n_cores_3_1 = job_row['n_cores_3_1']
    
    df = mean_core_consumptions[mean_core_consumptions['user_short'] == job_user]
    df = df[df['job_name'] == job_name]
    mean_core_cons = df['mean_overAll']
    
    job_predicted_consumption = job_n_cores_2_1 * mean_core_cons + job_n_cores_2_2 * mean_core_cons + job_n_cores_3_1 * mean_core_cons
    
    long_jobs.at[job_index, 'pred_userJob'] = job_predicted_consumption
    
    if (i % 10000 == 0):
        print(i)
    i += 1

0
10000
20000
30000


In [19]:
# write to long_jobs
long_jobs.to_csv(infile_long_jobs + ".csv")

# Validate

In [3]:
from sklearn.metrics import r2_score
from scipy import stats

from sklearn.metrics import mean_squared_error

good_quality_jobs = long_jobs[long_jobs['real_pow_quality'] != 0]
#good_quality_jobs = long_jobs


mean_real_pow = long_jobs['real_pow'].mean()
print("the mean is {}".format(mean_real_pow))


good_quality_jobs['numerator'] = (good_quality_jobs['real_pow'] - good_quality_jobs['pred_userJob'])**2
good_quality_jobs['denumerator'] = (good_quality_jobs['real_pow'] - mean_real_pow)**2
N = good_quality_jobs['numerator'].shape[0]

r_2 = 1 - (good_quality_jobs['numerator'].sum() / good_quality_jobs['denumerator'].sum())
print("r_2 is {}".format(r_2))

rmse = math.sqrt(good_quality_jobs['numerator'].sum() / N)
print("RMSE is {}".format(rmse))
print("NRMSE is {}".format(rmse/mean_real_pow))

the mean is 110.35204538702968
r_2 is 0.810136180971846
RMSE is 117.51841984031236
NRMSE is 1.0649410206049972


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [38]:
# Drop jobs with only one occurrence

grouped_jobs_names = long_jobs.groupby(['job_name'])['job_id_string'].nunique().reset_index(name='counts')

#print("{}".format(grouped_jobs_names.shape[0]))
grouped_jobs_names_multiple_occurrences = grouped_jobs_names[grouped_jobs_names['counts'] == 1]
print(grouped_jobs_names_multiple_occurrences.shape)
#print("{}".format(grouped_jobs_names_multiple_occurrences.shape[0]))
jobs_to_validate = long_jobs.merge(grouped_jobs_names, on=['job_name'])
print("{}/{}".format(jobs_to_validate.shape[0], long_jobs.shape[0]))

(1117, 2)
35284/35284


In [40]:
jobs_to_validate = jobs_to_validate[jobs_to_validate['counts'] > 1]
print("{}/{}".format(jobs_to_validate.shape[0], long_jobs.shape[0]))

34167/35284


In [41]:
jobs_to_validate[jobs_to_validate['job_name'] == '010-dos']

,job_id_string,job_name,queue,start_time,run_start_time,end_time,user,node_req,cpu_req,mem_req,...,n_3_1,pred_overAll,pred_perNode,n_cores_2_1,n_cores_2_2,n_cores_3_1,user_short,pred_user,pred_userJob,counts


In [43]:
#good_quality_jobs = long_jobs[long_jobs['real_pow_quality'] > 0.5]
good_quality_jobs = jobs_to_validate

slope, intercept, r_value, p_value, std_err = stats.linregress(good_quality_jobs['real_pow'], good_quality_jobs['pred_userJob'])
print("R-squared: {}".format(r_value**2))
from sklearn.metrics import mean_squared_error

MSE = mean_squared_error(good_quality_jobs['real_pow'], good_quality_jobs['pred_userJob'])
print("mean_quared_error: {}".format(MSE))
print(math.sqrt(MSE))
print(1-(math.sqrt(MSE)/(good_quality_jobs['real_pow'].mean() * good_quality_jobs['pred_userJob'].mean())))

R-squared: 0.7948459804467222
mean_quared_error: 14020.931815276912
118.41001568818793
0.9899108444480348
